In [2]:
import os
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style = 'white', font_scale=1.5)
from itertools import chain
from sklearn.linear_model import LogisticRegression

In [4]:
import sys
sys.path.insert(0, '/home/was966/Research/')
from responder.data import TCGAData, GeneData, ITRPData
from responder.aug import MixupNomralAugmentor
from responder.data import TCGAData, GeneData
from responder.model import TCGAPretrainModel
from responder.train import train, test, predict
from responder.saver import SaveBestModel
from responder.plot import plot_embed_with_label
from responder.trainer import TCGAPreTrainer, ITRPFineTuner
from responder.utils import score

/home/was966/micromamba/envs/RNA/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/was966/micromamba/envs/RNA/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/was966/micromamba/envs/RNA/lib/python3.11/site-packages/umap/distanc

In [5]:
def onehot(S):
    assert type(S) == pd.Series, 'Input type should be pd.Series'
    dfd = pd.get_dummies(S, dummy_na=True)
    nanidx = dfd[dfd[np.nan]].index
    dfd.loc[nanidx, :] = np.nan
    dfd = dfd.drop(columns=[np.nan])*1.
    return dfd
    
data_path = '/n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-MERGE/18425/'

### make prediction & plot on ITRP
itrp_df_tpm = pd.read_pickle(os.path.join(data_path, 'TPM.ITRP.TABLE'))
itrp_df_tpm = np.log2(itrp_df_tpm + 1)

itrp_df_meta = pd.read_pickle(os.path.join(data_path, 'PATIENT.ITRP.TABLE'))
itrp_df_meta['response_label'] = itrp_df_meta['Freeman_response']
itrp_df_meta['response_label'][itrp_df_meta.cohort == 'Gide'] = itrp_df_meta['RECIST_Response'][itrp_df_meta.cohort == 'Gide']

itrp_df_rps = itrp_df_meta[['response_label']]
itrp_df_cht = itrp_df_meta[['cohort']]
itrp_df_meta.groupby('cohort')['response_label'].value_counts().unstack().T
itrp_df_task = onehot(itrp_df_rps['response_label'])

def sample_validxy(train_X, train_y, frac=0.1):
    dft = train_y.idxmax(axis=1).to_frame('label')
    val_idx = dft.groupby('label').apply(lambda x:x.sample(frac=frac, 
                                                            random_state=42).index.tolist()).tolist()
    val_idx = list(chain(*val_idx))
    train_idx = dft[~dft.index.isin(val_idx)].index
    X_train = train_X.loc[train_idx]
    y_train = train_y.loc[train_idx]
    X_val = train_X.loc[val_idx]
    y_val = train_y.loc[val_idx]
    #print(len(X_train), len(X_val))
    return (X_train, y_train), (X_val,y_val)


dfx = itrp_df_tpm
dfy = itrp_df_task
dfc = itrp_df_cht

# dfx.to_csv('./R/data/ITRP_TPM.csv.gzip', compression ='gzip')
# dfy.to_csv('./R/data/ITRP_response_label.csv.gzip', compression ='gzip')
# dfc.to_csv('./R/data/ITRP_cohort.csv.gzip', compression ='gzip')

/tmp/ipykernel_185025/2875174609.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  itrp_df_meta['response_label'][itrp_df_meta.cohort == 'Gide'] = itrp_df_meta['RECIST_Response'][itrp_df_meta.cohort == 'Gide']


## NetBio

In [5]:
mk = pd.read_csv('NetBio200.PD1.cfg',sep='\t').gene_id.tolist()
dfx = itrp_df_tpm[mk]
dfy = itrp_df_task
dfc = itrp_df_cht

cohort_list = [ 'Allen', 'Riaz', 'Gide', 'Liu',  'Hugo', 'MGH'] #,  '
res = []
print(len(mk))
## Leave-cohort-out Test
for cohort_name in cohort_list:
    test_idx = dfx[dfc.cohort == cohort_name].index
    train_idx = dfc[~dfc.index.isin(test_idx)].index
    train_X = dfx.loc[train_idx]
    train_y = dfy.loc[train_idx]
    X_test = dfx.loc[test_idx]    
    y_test = dfy.loc[test_idx]
    (X_train, y_train), (X_val, y_val) = sample_validxy(train_X, train_y)

    model = LogisticRegression(C = 1)
    model.fit(train_X.values, train_y.values[:, 1])

    y_pred = model.predict(X_test.values)
    y_prob = model.predict_proba(X_test.values)[:, 1]
    y_true = y_test.values[:,1]

    s1 = score(pd.Series(y_true), pd.Series(y_prob), pd.Series(y_pred))
    roc, prc, f1, acc = s1
    res.append( [cohort_name, roc, prc, f1, acc] )
df_Netbio = pd.DataFrame(res, columns = ['cohort', 'roc', 'prc', 'f1', 'acc']).set_index('cohort').T

/home/was966/micromamba/envs/RNA/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/was966/micromamba/envs/RNA/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mo

200


/home/was966/micromamba/envs/RNA/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/was966/micromamba/envs/RNA/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mo

In [ ]:
TAM_MAKERS = {'marker':{'TAM': 'F13A1:FCER1A:CCL17:FOXQ1:ESPNL:CD1A:GATM:CCL13:PALLD:GALNT18:MAOA:RAMP1:STAB1:CCL26:CCL23:PARM1:CD1E:ITM2C:CALCRL:CRH:RGS18:MS4A6A:DHRS2:PON2:ALOX15:RAB33A:MOCOS:CCL18:IL17RB:FABP4:CMTM8:QPRT:CDR2L:DUOXA1:ABCC4:SYT17:PPP1R14A:PDGFC:GPT:FAM189A2:RASAL1:IPCEF1:ZNF366:MAP4K1:RAB30:PCED1B:TMIGD3:SH3BP4:RRS1:RNASE1'},
                 'marker_score_method': 'average', 
                 'marker_type':'set',
              'marker_ip':'Jiang',
                 'paper': ['PMID:30127393', 'PMID:35764641']}
mk = TAM_MAKERS['marker']['TAM'].split(':')
mk = list(set(mk) & set(itrp_df_tpm.columns))
print(len(mk))
dfx = itrp_df_tpm[mk]
dfy = itrp_df_task
dfc = itrp_df_cht

cohort_list = [ 'Allen', 'Riaz', 'Gide', 'Liu',  'Hugo', 'MGH'] #,  '
res = []

## Leave-cohort-out Test
for cohort_name in cohort_list:
    test_idx = dfx[dfc.cohort == cohort_name].index
    train_idx = dfc[~dfc.index.isin(test_idx)].index
    train_X = dfx.loc[train_idx]
    train_y = dfy.loc[train_idx]
    X_test = dfx.loc[test_idx]    
    y_test = dfy.loc[test_idx]
    (X_train, y_train), (X_val, y_val) = sample_validxy(train_X, train_y)

    model = LogisticRegression(C = 1)
    model.fit(train_X.values, train_y.values[:, 1])

    y_pred = model.predict(X_test.values)
    y_prob = model.predict_proba(X_test.values)[:, 1]
    y_true = y_test.values[:,1]

    s1 = score(pd.Series(y_true), pd.Series(y_prob), pd.Series(y_pred))
    roc, prc, f1, acc = s1
    res.append( [cohort_name, roc, prc, f1, acc] )
dfres = pd.DataFrame(res, columns = ['cohort', 'roc', 'prc', 'f1', 'acc']).set_index('cohort').T

In [ ]:
dfres

In [ ]:
GeneBio_MAKERS = {'marker':{'GeneBio': 'PDCD1:CD274:CTLA4'},
                 'marker_score_method': 'average', 
                 'marker_type':'set',
                  'marker_ip':'Kong',
                 'paper': ['PMID:30127393']}

mk = GeneBio_MAKERS['marker']['GeneBio'].split(':')
mk = list(set(mk) & set(itrp_df_tpm.columns))
print(len(mk))
dfx = itrp_df_tpm[mk]
dfy = itrp_df_task
dfc = itrp_df_cht

cohort_list = [ 'Allen', 'Riaz', 'Gide', 'Liu',  'Hugo', 'MGH'] #,  '
res = []

## Leave-cohort-out Test
for cohort_name in cohort_list:
    test_idx = dfx[dfc.cohort == cohort_name].index
    train_idx = dfc[~dfc.index.isin(test_idx)].index
    train_X = dfx.loc[train_idx]
    train_y = dfy.loc[train_idx]
    X_test = dfx.loc[test_idx]    
    y_test = dfy.loc[test_idx]
    (X_train, y_train), (X_val, y_val) = sample_validxy(train_X, train_y)

    model = LogisticRegression(C = 1)
    model.fit(train_X.values, train_y.values[:, 1])

    y_pred = model.predict(X_test.values)
    y_prob = model.predict_proba(X_test.values)[:, 1]
    y_true = y_test.values[:,1]

    s1 = score(pd.Series(y_true), pd.Series(y_prob), pd.Series(y_pred))
    roc, prc, f1, acc = s1
    res.append( [cohort_name, roc, prc, f1, acc] )
dfres = pd.DataFrame(res, columns = ['cohort', 'roc', 'prc', 'f1', 'acc']).set_index('cohort').T

In [ ]:
dfres

In [ ]:
Texh_MAKERS = {'marker':{'Texh': 'HSPA1B:HSPA1A:NR4A2:RGS1:TNFAIP3:CAMK2N1:DUSP1:NR4A3:IFIT3:IFIT1B:FOSB:CCL5:SAMD3:CCL3L3:MACC1:VPS37B:TF:DSEL:SPATA20:ADSSL1:DHX58:SPP1:TRIM15:DUSP26:ABI3:SFN:C1QC:RTP4:JUN:BCKDHB:CCL4:POU6F1:CYSLTR2:SLC14A1:RGS16:CUEDC1:DEDD2:CPEB1:CTSS:TMEM88:FAM196B:SKI:BTG1:SYCP2L:RGS2:ABCB1:ADRB2:CD69:RNF166:TNN'},
               'marker_score_method': 'average', 
               'marker_type':'set',
               'marker_ip':'Jiang',
               'paper': ['PMID:30127393', 'PMID:35764641']}

mk = Texh_MAKERS['marker']['Texh'].split(':')
mk = list(set(mk) & set(itrp_df_tpm.columns))
print(len(mk))
dfx = itrp_df_tpm[mk]
dfy = itrp_df_task
dfc = itrp_df_cht

cohort_list = [ 'Allen', 'Riaz', 'Gide', 'Liu',  'Hugo', 'MGH'] #,  '
res = []

## Leave-cohort-out Test
for cohort_name in cohort_list:
    test_idx = dfx[dfc.cohort == cohort_name].index
    train_idx = dfc[~dfc.index.isin(test_idx)].index
    train_X = dfx.loc[train_idx]
    train_y = dfy.loc[train_idx]
    X_test = dfx.loc[test_idx]    
    y_test = dfy.loc[test_idx]
    (X_train, y_train), (X_val, y_val) = sample_validxy(train_X, train_y)

    model = LogisticRegression(C = 1)
    model.fit(train_X.values, train_y.values[:, 1])

    y_pred = model.predict(X_test.values)
    y_prob = model.predict_proba(X_test.values)[:, 1]
    y_true = y_test.values[:,1]

    s1 = score(pd.Series(y_true), pd.Series(y_prob), pd.Series(y_pred))
    roc, prc, f1, acc = s1
    res.append( [cohort_name, roc, prc, f1, acc] )
dfres = pd.DataFrame(res, columns = ['cohort', 'roc', 'prc', 'f1', 'acc']).set_index('cohort').T

In [1]:
dfres

NameError: name 'dfres' is not defined

In [10]:
PGM_MAKERS = {'marker':{'PGM1':'MAP4K1:TBX3', 'PGM2':'MAP4K1:AGER'},
              'marker_score_method':'diff', 
              'marker_type':'pair',
              'marker_ip': 'Freeman',
              'paper': ['PMID: 35243413']}

mk = PGM_MAKERS['marker']['PGM2'].split(':')
mk = list(set(mk) & set(itrp_df_tpm.columns))
print(len(mk))
dfx = itrp_df_tpm[mk]
dfy = itrp_df_task
dfc = itrp_df_cht

cohort_list = [ 'Allen', 'Riaz', 'Gide', 'Liu',  'Hugo', 'MGH'] #,  '
res = []

## Leave-cohort-out Test
for cohort_name in cohort_list:
    test_idx = dfx[dfc.cohort == cohort_name].index
    train_idx = dfc[~dfc.index.isin(test_idx)].index
    train_X = dfx.loc[train_idx]
    train_y = dfy.loc[train_idx]
    X_test = dfx.loc[test_idx]    
    y_test = dfy.loc[test_idx]
    (X_train, y_train), (X_val, y_val) = sample_validxy(train_X, train_y)

    model = LogisticRegression(C = 1)
    model.fit(train_X.values, train_y.values[:, 1])

    y_pred = model.predict(X_test.values)
    y_prob = model.predict_proba(X_test.values)[:, 1]
    y_true = y_test.values[:,1]

    s1 = score(pd.Series(y_true), pd.Series(y_prob), pd.Series(y_pred))
    roc, prc, f1, acc = s1
    res.append( [cohort_name, roc, prc, f1, acc] )
dfres = pd.DataFrame(res, columns = ['cohort', 'roc', 'prc', 'f1', 'acc']).set_index('cohort').T

2


In [11]:
dfres.mean(axis=1)

roc    0.677278
prc    0.598492
f1     0.420748
acc    0.639309
dtype: float64

In [ ]:


PGD_MAKERS = {'marker':{'PGD1':'SLAMF6:ATG12', 
                        'PGD2':'SLAMF7:IL10',
                        'PGD3':'PIK3CD:ATG12',
                        'PGD4':'DEFB110:SFTPA2',
                        'PGD5':'IKZF3:APAF1',
                        'PGD6':'IKZF3:CD200R1',
                        'PGD7':'IL6R:RER1',
                        'PGD8':'IL6R:CTNNA1'},
              
              'marker_score_method':'diff', 
              'marker_type':'pair',
              'marker_ip': 'Shen',
              'paper': ['N.A.']}


mk = PGD_MAKERS['marker']['PGD2'].split(':')
mk = list(set(mk) & set(itrp_df_tpm.columns))
print(len(mk))
dfx = itrp_df_tpm[mk]
dfy = itrp_df_task
dfc = itrp_df_cht

cohort_list = [ 'Allen', 'Riaz', 'Gide', 'Liu',  'Hugo', 'MGH'] #,  '
res = []

## Leave-cohort-out Test
for cohort_name in cohort_list:
    test_idx = dfx[dfc.cohort == cohort_name].index
    train_idx = dfc[~dfc.index.isin(test_idx)].index
    train_X = dfx.loc[train_idx]
    train_y = dfy.loc[train_idx]
    X_test = dfx.loc[test_idx]    
    y_test = dfy.loc[test_idx]
    (X_train, y_train), (X_val, y_val) = sample_validxy(train_X, train_y)

    model = LogisticRegression(C = 1)
    model.fit(train_X.values, train_y.values[:, 1])

    y_pred = model.predict(X_test.values)
    y_prob = model.predict_proba(X_test.values)[:, 1]
    y_true = y_test.values[:,1]

    s1 = score(pd.Series(y_true), pd.Series(y_prob), pd.Series(y_pred))
    roc, prc, f1, acc = s1
    res.append( [cohort_name, roc, prc, f1, acc] )
dfres = pd.DataFrame(res, columns = ['cohort', 'roc', 'prc', 'f1', 'acc']).set_index('cohort').T

In [ ]:
dfres

In [12]:
MARKERS = ['CXCL10', 'IFNG', 'CXCL9', 'IDO1', 'HLA-DRA', 'STAT1']
MARKERS_EIGS = ['CD3D','IDO1','CIITA','CD3E','CCL5','GZMK','CD2','HLA-DRA','CXCL13',
              'IL2RG','NKG7','HLA-E','CXCR6','LAG3','TAGAP','CXCL10','STAT1','GZMB', ]

mk = MARKERS
print(list(set(mk) - set(itrp_df_tpm.columns)))

mk = list(set(mk) & set(itrp_df_tpm.columns))

print(len(mk))
dfx = itrp_df_tpm[mk]
dfy = itrp_df_task
dfc = itrp_df_cht

cohort_list = [ 'Allen', 'Riaz', 'Gide', 'Liu',  'Hugo', 'MGH'] #,  '
res = []

## Leave-cohort-out Test
for cohort_name in cohort_list:
    test_idx = dfx[dfc.cohort == cohort_name].index
    train_idx = dfc[~dfc.index.isin(test_idx)].index
    train_X = dfx.loc[train_idx]
    train_y = dfy.loc[train_idx]
    X_test = dfx.loc[test_idx]    
    y_test = dfy.loc[test_idx]
    (X_train, y_train), (X_val, y_val) = sample_validxy(train_X, train_y)

    model = LogisticRegression(C = 1)
    model.fit(train_X.values, train_y.values[:, 1])

    y_pred = model.predict(X_test.values)
    y_prob = model.predict_proba(X_test.values)[:, 1]
    y_true = y_test.values[:,1]

    s1 = score(pd.Series(y_true), pd.Series(y_prob), pd.Series(y_pred))
    roc, prc, f1, acc = s1
    res.append( [cohort_name, roc, prc, f1, acc] )
dfres = pd.DataFrame(res, columns = ['cohort', 'roc', 'prc', 'f1', 'acc']).set_index('cohort').T

[]
6


In [9]:
dfres

cohort,Allen,Riaz,Gide,Liu,Hugo,MGH
roc,0.636095,0.536585,0.718182,0.504804,0.446429,0.651786
prc,0.457227,0.335156,0.681295,0.423942,0.566991,0.585007
f1,0.400000,0.300000,0.647059,0.366197,0.133333,0.380952
acc,0.692308,0.725490,0.671233,0.579439,0.500000,0.657895


In [11]:
dfres

cohort,Allen,Riaz,Gide,Liu,Hugo,MGH
roc,0.579882,0.639024,0.453788,0.509608,0.464286,0.473214
prc,0.419401,0.338380,0.506830,0.361739,0.579044,0.450657
f1,0.272727,0.341463,0.000000,0.111111,0.133333,0.000000
acc,0.589744,0.470588,0.452055,0.551402,0.500000,0.631579


In [72]:
from tidepy.pred import TIDE, MODEL_DB

NAME = 'TIDE'


dfx = itrp_df_tpm
dfy = itrp_df_task
dfc = itrp_df_cht


def get_tide(df_tpm, cancer='Melanoma',):
    #df_tpm_log2 = np.log2(df_tpm + 1)
    #control = df_tpm_log2.mean()
    #df_tpm_norm = df_tpm_log2 - control 
    #df_tpm_input = df_tpm_norm.T
    df_tpm_input = df_tpm.T
    result = TIDE(df_tpm_input, cancer=cancer,pretreat=True,force_normalize = True, vthres=0.)

    '''
    ['No benefits', 'Responder', 'TIDE', 'IFNG', 'MSI Score', 'CD274', 'CD8',
    'CTL.flag', 'Dysfunction', 'Exclusion', 'MDSC', 'CAF', 'TAM M2', 'CTL']
    '''
    return result


cohort_list = [ 'Allen', 'Riaz', 'Gide', 'Liu',  'Hugo', 'MGH'] #,  '
res = []

## Leave-cohort-out Test
for cohort_name in cohort_list:
    test_idx = dfx[dfc.cohort == cohort_name].index
    train_idx = dfc[~dfc.index.isin(test_idx)].index
    train_X = dfx.loc[train_idx]
    train_y = dfy.loc[train_idx]
    X_test = dfx.loc[test_idx]    
    y_test = dfy.loc[test_idx]

    y_pred = get_tide(X_test).Responder.values*1.
    y_prob = get_tide(X_test).TIDE.values
    y_true = y_test.values[:,1]

    s1 = score(pd.Series(y_true), pd.Series(y_prob), pd.Series(y_pred))
    roc, prc, f1, acc = s1
    res.append( [cohort_name, roc, prc, f1, acc] )
dfres = pd.DataFrame(res, columns = ['cohort', 'roc', 'prc', 'f1', 'acc']).set_index('cohort').T

[WARN] 4.66 % Genes are missing after converting to Entrez ID
[WARN] The majority(>80%) of genes with positive expression in your inputted data. Please Normalize your data
[WARN] Start normalizing the input expression profile by: 1. Do the log2(x+1) transformation. 2. Subtract the average across your samples.
[WARN] 4.66 % Genes are missing after converting to Entrez ID
[WARN] The majority(>80%) of genes with positive expression in your inputted data. Please Normalize your data
[WARN] Start normalizing the input expression profile by: 1. Do the log2(x+1) transformation. 2. Subtract the average across your samples.
[WARN] 4.66 % Genes are missing after converting to Entrez ID
[WARN] The majority(>80%) of genes with positive expression in your inputted data. Please Normalize your data
[WARN] Start normalizing the input expression profile by: 1. Do the log2(x+1) transformation. 2. Subtract the average across your samples.
[WARN] 4.66 % Genes are missing after converting to Entrez ID
[WARN

In [74]:
dfres

cohort,Allen,Riaz,Gide,Liu,Hugo,MGH
roc,0.381657,0.485366,0.297727,0.660754,0.547619,0.482143
prc,0.352064,0.182918,0.412459,0.579213,0.583259,0.429706
f1,0.516129,0.285714,0.693333,0.315789,0.444444,0.411765
acc,0.615385,0.509804,0.684932,0.392523,0.423077,0.473684
